In [23]:
import glob
import pandas as pd

covid_data = 'Data/covid/*.csv'
vaccination_usa = 'Data/vaccinations/us.csv'
vaccination_usa_state = 'Data/vaccinations/us_state_vaccinations.csv'

In [15]:
data = pd.DataFrame()

# Read all css files per day comnine in one dataset
for file in glob.glob(covid_data):
    df = pd.read_csv(file)
    data = data.append(df,ignore_index=True)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate,Total_Test_Results,Case_Fatality_Ratio
0,Alabama,US,2020-06-08 03:33:42,32.3182,-86.9023,20777,692,11395.0,8413.0,1.0,418.095585,259566.0,2022.0,3.375610,84000001.0,USA,5293.824320,9.863415,NaN,NaN
1,Alaska,US,2020-06-08 03:33:42,61.3707,-152.4044,543,10,382.0,151.0,2.0,74.226466,64904.0,NaN,1.841621,84000002.0,USA,8872.181479,NaN,NaN,NaN
2,American Samoa,US,2020-06-08 03:33:42,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,174.0,NaN,NaN,16.0,ASM,312.719038,NaN,NaN,NaN
3,Arizona,US,2020-06-08 03:33:42,33.7298,-111.4312,26989,1051,5517.0,20421.0,4.0,370.793369,281621.0,3352.0,3.894179,84000004.0,USA,3869.102206,12.419875,NaN,NaN
4,Arkansas,US,2020-06-08 03:33:42,34.9697,-92.3731,9426,154,6424.0,2848.0,5.0,312.346329,160273.0,844.0,1.633779,84000005.0,USA,5310.914824,8.953957,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,Washington,US,2020-04-25 06:32:46,47.4009,-121.4905,12977,762,NaN,12255.0,53.0,171.894387,160324.0,455.0,5.563690,84000053.0,USA,2123.664613,3.506203,NaN,NaN
17994,West Virginia,US,2020-04-25 06:32:46,38.4912,-80.9545,1010,32,439.0,978.0,54.0,76.341763,29811.0,97.0,3.168317,84000054.0,USA,2253.291388,9.603960,NaN,NaN
17995,Wisconsin,US,2020-04-25 06:32:46,44.2685,-89.6165,5356,263,NaN,5093.0,55.0,103.506165,59929.0,1353.0,4.910381,84000055.0,USA,1158.144318,25.261389,NaN,NaN
17996,Wyoming,US,2020-04-25 06:32:46,42.7560,-107.3025,473,7,321.0,466.0,56.0,95.125090,8045.0,54.0,1.479915,84000056.0,USA,1617.930979,11.416490,NaN,NaN


In [39]:
# Convert data to YYYY-MM-DD format
data['Last_Update'] = pd.to_datetime(data['Last_Update'], format='%Y/%m/%d %H:%M:%S').dt.strftime('%Y-%m-%d')
data


,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate,Total_Test_Results,Case_Fatality_Ratio
0,Alabama,US,2020-06-08,32.3182,-86.9023,20777,692,11395.0,8413.0,1.0,418.095585,259566.0,2022.0,3.375610,84000001.0,USA,5293.824320,9.863415,NaN,NaN
1,Alaska,US,2020-06-08,61.3707,-152.4044,543,10,382.0,151.0,2.0,74.226466,64904.0,NaN,1.841621,84000002.0,USA,8872.181479,NaN,NaN,NaN
2,American Samoa,US,2020-06-08,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,174.0,NaN,NaN,16.0,ASM,312.719038,NaN,NaN,NaN
3,Arizona,US,2020-06-08,33.7298,-111.4312,26989,1051,5517.0,20421.0,4.0,370.793369,281621.0,3352.0,3.894179,84000004.0,USA,3869.102206,12.419875,NaN,NaN
4,Arkansas,US,2020-06-08,34.9697,-92.3731,9426,154,6424.0,2848.0,5.0,312.346329,160273.0,844.0,1.633779,84000005.0,USA,5310.914824,8.953957,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,Washington,US,2020-04-25,47.4009,-121.4905,12977,762,NaN,12255.0,53.0,171.894387,160324.0,455.0,5.563690,84000053.0,USA,2123.664613,3.506203,NaN,NaN
17994,West Virginia,US,2020-04-25,38.4912,-80.9545,1010,32,439.0,978.0,54.0,76.341763,29811.0,97.0,3.168317,84000054.0,USA,2253.291388,9.603960,NaN,NaN
17995,Wisconsin,US,2020-04-25,44.2685,-89.6165,5356,263,NaN,5093.0,55.0,103.506165,59929.0,1353.0,4.910381,84000055.0,USA,1158.144318,25.261389,NaN,NaN
17996,Wyoming,US,2020-04-25,42.7560,-107.3025,473,7,321.0,466.0,56.0,95.125090,8045.0,54.0,1.479915,84000056.0,USA,1617.930979,11.416490,NaN,NaN


In [38]:
data.isna().sum()

Province_State              0
Country_Region              0
Last_Update                19
Lat                       638
Long_                     638
Confirmed                   0
Deaths                      0
Recovered                3591
Active                     17
FIPS                       19
Incident_Rate             638
People_Tested            6182
People_Hospitalized     12869
Mortality_Rate           5971
UID                         0
ISO3                        0
Testing_Rate              638
Hospitalization_Rate    12869
Total_Test_Results      12454
Case_Fatality_Ratio     12355
dtype: int64

In [20]:
data[data['Last_Update'].str.contains('2021-02-10') == True]

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate,Total_Test_Results,Case_Fatality_Ratio
4818,Alabama,US,2021-02-10 05:30:29,32.3182,-86.9023,474666,8579,252880.0,213207.0,1.0,9680.768725,NaN,NaN,NaN,84000001.0,USA,44824.150017,NaN,2197811.0,1.807376
4819,Alaska,US,2021-02-10 05:30:29,61.3707,-152.4044,55893,280,NaN,55613.0,2.0,7640.404896,NaN,NaN,NaN,84000002.0,USA,213418.039902,NaN,1561249.0,0.500957
4820,American Samoa,US,2021-02-10 05:30:29,-14.2710,-170.1320,0,0,NaN,0.0,60.0,0.000000,NaN,NaN,NaN,16.0,ASM,3846.084722,NaN,2140.0,NaN
4821,Arizona,US,2021-02-10 05:30:29,33.7298,-111.4312,787268,14286,108965.0,664017.0,4.0,10816.027055,NaN,NaN,NaN,84000004.0,USA,49043.890565,NaN,3569766.0,1.814630
4822,Arkansas,US,2021-02-10 05:30:29,34.9697,-92.3731,308848,5148,288774.0,14926.0,5.0,10234.196787,NaN,NaN,NaN,84000005.0,USA,84247.817287,NaN,2542434.0,1.666839
4823,California,US,2021-02-10 08:30:40,36.1162,-119.6816,3442672,45009,NaN,3397663.0,6.0,8712.929161,NaN,NaN,NaN,84000006.0,USA,112834.208290,NaN,44583304.0,1.307386
4824,Colorado,US,2021-02-10 05:30:29,39.0598,-105.3111,407210,5746,21611.0,379853.0,8.0,7071.169784,NaN,NaN,NaN,84000008.0,USA,98713.172474,NaN,5684631.0,1.411066
4825,Connecticut,US,2021-02-10 05:30:29,41.5978,-72.7554,264608,7298,9800.0,247510.0,9.0,7421.786801,NaN,NaN,NaN,84000009.0,USA,171221.896021,NaN,6104552.0,2.758042
4826,Delaware,US,2021-02-10 05:30:29,39.3185,-75.5071,81210,1245,NaN,79989.0,10.0,8339.803073,NaN,NaN,NaN,84000010.0,USA,132045.649665,NaN,1285813.0,1.503509
4827,Diamond Princess,US,2021-02-10 05:30:29,NaN,NaN,49,0,NaN,49.0,88888.0,NaN,NaN,NaN,NaN,84088888.0,USA,NaN,NaN,NaN,0.000000


In [11]:
data['Last_Update'].str.match('2021-01-20')

0        False
1        False
2        False
3        False
4        False
         ...  
17993    False
17994    False
17995    False
17996    False
17997    False
Name: Last_Update, Length: 17998, dtype: object

In [25]:
vac_usa_data = pd.read_csv(vaccination_usa)
vac_usa_data

,location,date,vaccine,source_url,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,United States,2020-12-20,Pfizer/BioNTech,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,556208,556208.0,NaN
1,United States,2020-12-21,Pfizer/BioNTech,https://covid.cdc.gov/covid-data-tracker/#vacc...,614117,614117.0,NaN
2,United States,2020-12-23,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1008025,1008025.0,NaN
3,United States,2020-12-26,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,1944585,1944585.0,NaN
4,United States,2020-12-28,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2127143,2127143.0,NaN
5,United States,2020-12-30,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,2794588,2794588.0,NaN
6,United States,2021-01-02,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4225756,4225756.0,NaN
7,United States,2021-01-04,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4563260,4563260.0,NaN
8,United States,2021-01-05,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,4836469,4836469.0,NaN
9,United States,2021-01-06,"Moderna, Pfizer/BioNTech",https://covid.cdc.gov/covid-data-tracker/#vacc...,5306797,5306797.0,NaN


In [42]:
vac_state_data = pd.read_csv(vaccination_usa_state)
vac_state_data[vac_state_data['location'] == 'Texas']

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
1944,2021-01-12,Texas,911461.0,1949125.0,809148.0,0.35,3.14,101317.0,2.79,6.72,NaN,NaN,NaN,0.468
1945,2021-01-13,Texas,978042.0,1984225.0,861072.0,0.40,3.37,115965.0,2.97,6.84,66581.00,66581.0,2296.0,0.493
1946,2021-01-14,Texas,1043624.0,2105600.0,917387.0,NaN,3.60,NaN,3.16,7.26,65582.00,66082.0,2279.0,0.496
1947,2021-01-15,Texas,1160242.0,2105600.0,1003412.0,0.54,4.00,155660.0,3.46,7.26,116618.00,82927.0,2860.0,0.551
1948,2021-01-16,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60936.75,77429.0,2670.0,NaN
1949,2021-01-17,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60936.75,74131.0,2557.0,NaN
1950,2021-01-18,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60936.75,71932.0,2481.0,NaN
1951,2021-01-19,Texas,1403989.0,2105600.0,1222952.0,0.62,4.84,179801.0,4.22,7.26,60936.75,70361.0,2427.0,0.667
1952,2021-01-20,Texas,1471746.0,2551125.0,1282998.0,0.65,5.08,187503.0,4.42,8.80,67757.00,70529.0,2432.0,0.577
1953,2021-01-21,Texas,1544551.0,2754800.0,1337706.0,0.71,5.33,205538.0,4.61,9.50,72805.00,71561.0,2468.0,0.561


In [40]:
vac_state_data.isna().sum()

date                                     0
location                                 0
total_vaccinations                     270
total_distributed                      312
people_vaccinated                      324
people_fully_vaccinated_per_hundred    532
total_vaccinations_per_hundred         430
people_fully_vaccinated                406
people_vaccinated_per_hundred          455
distributed_per_hundred                443
daily_vaccinations_raw                  65
daily_vaccinations                      65
daily_vaccinations_per_million         240
share_doses_used                       312
dtype: int64